<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/LLM/training/examples_llama3_1_train_with_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Unsloth comes with pre-quantitized models. Please find this models at https://huggingface.co/unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Add a LORA Adapter to the pre-trained model

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context and 30% less VRAM
    random_state = 3407,
    use_rslora = False,  #  rank stabilized LoRA is off but True it is supported by unsloth
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


process own training set and implement a prompt generator that generates prompt for every row of the dataset

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

After processing the dataset has 4 columns. The added column "text" contains the generated prompt that is used for training

In [ ]:
print (dataset)

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 51760
})


after we prepared our dataset we can use a standard SFTTrainer from huggingface to ("fine-tune") train our unsloth model. The unsloth model is just pass in to training as any other CausalLLM model.   

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


Now we start the training process

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.817800
2,2.149900
3,1.704600
4,1.909400
5,1.630300
6,1.542500
7,1.125200
8,1.235900
9,1.069800
10,1.138400


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

437.6269 seconds used for training.
7.29 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


Now we can use this model for interference. Textstreamer just output every new token instead of waiting for the whole result. But can also be done with the TextStreamer

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5772865, 9275249, 15049562, 24409871, 39689433, 643489


Now save to LoRA Adapter

In [ ]:
model.save_pretrained("llama3.1_alpaca_lora_adapter") # Local saving
tokenizer.save_pretrained("llama3.1_alpaca_lora_adapter")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# Merge to 16bit
if False: model.save_pretrained_merged("llama3.1_alpaca_16bit_model", tokenizer, save_method = "merged_16bit",)
if False: model.save_pretrained_merged("llama3.1_alpaca_4bit_model", tokenizer, save_method = "merged_4bit",)
if True: model.save_pretrained_gguf("llama3.1_alpaca_llama_cpp_q4_model", tokenizer, quantization_method = "q4_k_m")  ## see other quantization methods

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.79 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:37<00:00,  3.03s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at llama3.1_alpaca_llama_cpp_q4_model into f16 GGUF format.
The output location will be ./llama3.1_alpaca_llama_cpp_q4_model/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: llama3.1_alpaca_llama_cpp_q4_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F1

In [ ]:
!zip -r llama3.1_alpaca_llama_cpp_q4_model.zip llama3.1_alpaca_llama_cpp_q4_model/

  adding: llama3.1_alpaca_llama_cpp_q4_model/ (stored 0%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00002-of-00004.bin (deflated 9%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/pytorch_model.bin.index.json (deflated 95%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00001-of-00004.bin (deflated 12%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/unsloth.Q4_K_M.gguf (deflated 2%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/generation_config.json (deflated 38%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/unsloth.F16.gguf (deflated 11%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/special_tokens_map.json (deflated 71%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00003-of-00004.bin (deflated 9%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/tokenizer.json (deflated 74%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/tokenizer_config.json (deflated 94%)
  adding: llama3.1_alpaca_llama_cpp_q4_model/pytorch_model-00004-of-00004.bin (deflated 22%)
  a

we can load the model Lora adapter for interference

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "llama3.1_alpaca_lora_adapter", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

To run this locally try GPT4all https://www.nomic.ai/gpt4all